In [ ]:
import ants
import numpy as np

In [ ]:
fname1 = ants.get_ants_data('r16')
fname2 = ants.get_ants_data('r64')
print(fname1)
print(fname2)

In [ ]:
img1 = ants.image_read(fname1)
img2 = ants.image_read(fname2)
print(img1)

In [ ]:
arr_4d = np.random.randn(70,70,70,10).astype('float32')
img_fmri = ants.from_numpy(arr_4d, has_components=True)
print(img_fmri)

# todo : tester aussi from_nibabel/ to_nibabel (-> .nii.gz)
#        nifti_to_ants

In [ ]:
img = ants.image_read(ants.get_ants_data('r16'))
img = ants.resample_image(img, (64,64), 1, 0)
mask = ants.get_mask(img)
img_seg = ants.atropos(a=img, 
                       m='[0.2,1x1]', 
                       c='[2,0]', 
                       i='kmeans[3]', 
                       x=mask)
print(img_seg.keys())
ants.plot(img_seg['segmentation'])

In [ ]:
mg = ants.image_read( ants.get_ants_data('r16') ,2)
mask = ants.get_mask( img ).threshold_image( 1, 2 )
segs=ants.atropos( a = img, 
                   m = '[0.2,1x1]', 
                   c = '[2,0]',  
                   i = 'kmeans[3]', 
                   x = mask )
thickimg = ants.kelly_kapowski(s=segs['segmentation'], 
                               g=segs['probabilityimages'][1],
                               w=segs['probabilityimages'][2], 
                               its=45, 
                               r=0.5, 
                               m=1)
print(thickimg)
img.plot(overlay=thickimg, overlay_cmap='jet')

# Transform manually template 

In [ ]:
import nrrd
import nibabel as nib
import numpy as np
import os
from pathlib import Path

template_file = '/Users/mahdi/stage-2022-mahdi/src/allen_template_25.nii.gz'
template_ras_file = "/Users/mahdi/stage-2022-mahdi/src/allen_template_25_manualRAS.nii.gz"

resolution = 25
res_avgt_mm = 70 / 1e3
avgt_offset = np.array([-5.675, -8.79448, -8.450335, 0])

# Load and manually transform the template
vol = nib.load(template_file).get_fdata()

vol_ras = np.moveaxis(vol, (0,1,2), (1,2,0))
vol_ras = np.flip(vol_ras, axis=2)
vol_ras = np.flip(vol_ras, axis=1)

affine = np.eye(4) * res_avgt_mm

img_ras = nib.Nifti1Image(vol_ras, affine)
nib.save(img_ras, template_ras_file)

# load reference
avgt = nib.load('/Users/mahdi/stage-2022-mahdi/src/AVGT.nii.gz')
ref_vol = avgt.get_fdata()

# load moving
allen = nib.load('/Users/mahdi/stage-2022-mahdi/src/allen_template_25_manualRAS.nii.gz')
allen_vol = allen.get_fdata()


allen_vol = allen_vol.astype(float)
ref_vol = ref_vol.astype(float)

fixed = ants.from_numpy( ref_vol ).resample_image((164, 212, 158),1,0)
moving = ants.from_numpy( allen_vol ).resample_image((164, 212, 158),1,0)

fixed.plot_ortho()
moving.plot_ortho()
mytx = ants.registration(fixed=fixed , moving=moving, type_of_transform='SyN' )

warped_moving = ants.apply_transforms(fixed=fixed, moving=moving, 
                                          transformlist=mytx['fwdtransforms'])
fixed.plot_ortho_double(warped_moving)
affine_offset = affine.copy()
affine_offset[:,3] + avgt_offset

warped_vol = warped_moving.numpy()
# Saving the volume (nifti)
img = nib.Nifti1Image(warped_vol, affine_offset)
nib.save(img, '/Users/mahdi/stage-2022-mahdi/src/template_reg.nii.gz')

In [ ]:
warped_moving.shape

### essais nii nibabel ants

In [ ]:
# todo : tester aussi from_nibabel/ to_nibabel (-> .nii.gz)
#        nifti_to_ants

import ants
import nrrd
import numpy as np

array, metadata = nrrd.read('/Users/mahdi/stage-2022-mahdi/src/average_template_25.nrrd')

img = ants.from_numpy(array) # convertir en float

orientation = ants.get_orientation(img)

print(orientation)

In [ ]:
import ants
import numpy as np
import matplotlib.pyplot as plt

img = ants.image_read(ants.get_ants_data('r16'))
mask = ants.get_mask(img, cleanup=15, low_thresh=55)
ants.plot(mask)
nimg = img * mask
ants.plot(nimg,black_bg=False)


In [ ]:
import numpy as np
import nibabel as nib


ants.ANTsImage.pixeltype = np.int32
img = nib.load('/Users/mahdi/stage-2022-mahdi/src/288169135_projection_density_50.nii.gz')

vol = img.get_fdata().astype(np.float32)

img = ants.from_numpy( vol ).resample_image((164, 212, 158),1,0)

img.plot_ortho()

for z in range(158):
    for y in range(212):
        for x in range(164):
            if vol[x, y, z] < 1e-01:
                vol[x, y, z] = 0.0

img = ants.from_numpy( vol )

img.plot_ortho()

 # Removing background 
    # mask = ants.get_mask(warped_moving)
    # warped_moving = warped_moving * mask



#####BINARIZED FORMULA######    S
    #warped_moving = warped_moving > 1e-01

In [ ]:
import cv2

tmp = cv2.cvtColor(img.numpy(), cv2.COLOR_BGR2GRAY)
_,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
b, g, r = cv2.split(img.numpy())
rgba = [b,g,r, alpha]
dst = cv2.merge(rgba,4)


In [ ]:
from PIL import Image
import numpy as np

image = Image.open('../images/288169135_proj_density_/288169266.jpg')

vol = np.asarray(image)

img = nib.Nifti1Image(vol, np.eye(4))

nib.save(img, '../../niftis/test.nii')

# test fusions 2 niftis

In [1]:
import ants
import numpy as np
import nibabel as nib
import PIL

In [17]:
vol = nib.load('/Users/mahdi/stage-2022-mahdi/niftis/485847695_AIp_R_proj_density_100.nii.gz').get_fdata()
voll = nib.load('/Users/mahdi/stage-2022-mahdi/niftis/180403712_ECT_R_proj_density_100.nii.gz').get_fdata()

In [11]:
print(vol[109,97,117],
      voll[109,97,117])   

0.0 0.0004686765605583787


In [19]:
print(vol[117,69,76],
      voll[117,69,76])

0.00012096612044842914 0.0014684608904644847


In [18]:
rgb = np.zeros((164,212,158,1,1), [('R', 'u1'), ('G','u1'), ('B', 'u1'), ('A', 'u1')])

for i in range(164):
    for j in range(212):
        for k in range(158):
            if vol[i,j,k] == 0.0 and voll[i,j,k] == 0.0:
                rgb[i,j,k] = (0, 0, 0, 0)
            else:
                rgb[i,j,k] = (vol[i,j,k] * 255, voll[i,j,k] * 255, 0, 255)

img_ = nib.Nifti1Image(rgb, affine)
nib.save(img_, '/Users/mahdi/stage-2022-mahdi/niftis/rbg_180601025_183171679.nii.gz')

In [21]:
print(rgb[117,69,76])

[[(0, 0, 0, 255)]]


In [3]:
vol1 = nib.load('/Users/mahdi/stage-2022-mahdi/niftis/288169135_MOp_R_proj_density_25.nii.gz').get_fdata()
vol2 = nib.load('/Users/mahdi/stage-2022-mahdi/niftis/100141563_MOp_R_proj_density_25.nii.gz').get_fdata()
vol3 = nib.load('/Users/mahdi/stage-2022-mahdi/niftis/603468246_ENTm_L_proj_density_25.nii.gz').get_fdata()

v4 = nib.load('/Users/mahdi/Downloads/180601025_SSp-un_R_proj_density_25.nii.gz').get_fdata()
v5 = nib.load('/Users/mahdi/Downloads/183171679_SSp-bfd_R_proj_density_25.nii.gz').get_fdata()

In [15]:
affine = nib.load('/Users/mahdi/stage-2022-mahdi/utils/AVGT.nii.gz').affine

In [4]:
rgb = np.zeros((164,212,158,1,1), [('R', 'u1'), ('G','u1'), ('B', 'u1'), ('A', 'u1')])

for i in range(164):
    for j in range(212):
        for k in range(158):
            if v4[i,j,k] == 0 and v5[i,j,k] == 0:
                rgb[i,j,k] = (0, 0, 0, 0)
            else:
                rgb[i,j,k] = (v5[i,j,k] * 255, v4[i,j,k] * 255, 0, 255)

img_ = nib.Nifti1Image(rgb, affine)
nib.save(img_, '/Users/mahdi/stage-2022-mahdi/niftis/rbg_180601025_183171679.nii.gz')

## Test creation volume RGB

In [5]:
arr = np.zeros((164,212,158,1,1), [('R', 'u1'), ('G','u1'), ('B', 'u1'), ('A', 'u1')])

for i in range(164):
    for j in range(212):
        for k in range(158):
            if vol1[i,j,k] == 0 and vol2[i,j,k] == 0:
                arr[i,j,k] = (0, 0, 0, 0)
            else:
                arr[i,j,k] = (vol1[i,j,k] * 255, vol2[i,j,k] * 255, 0, 255)


In [6]:
print(arr[101,127,118,0,0])
print(arr[51,103,65,0,0])

(255, 62, 0, 255)
(0, 0, 0, 0)


In [7]:
img = nib.Nifti1Image(arr, affine)
nib.save(img, '/Users/mahdi/stage-2022-mahdi/niftis/rbg.nii.gz')

In [150]:
fname = '/Users/mahdi/stage-2022-mahdi/niftis/rbg.nii.gz'
im = nib.load(fname)
print(im.get_data_dtype())
print(im.header)

[('R', 'u1'), ('G', 'u1'), ('B', 'u1'), ('A', 'u1')]
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  5 164 212 158   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : RGBA
bitpix          : 32
slice_start     : 0
pixdim          : [1.         0.06999999 0.06999999 0.06999999 1.         1.
 1.         1.        ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : unknown
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 0.0
qoffset_x     

# Mosaic

In [2]:
import ants
import numpy as np
import nibabel as nib

In [3]:
template = nib.load('/Users/mahdi/stage-2022-mahdi/niftis/allen_volume_100.nii.gz')
xroi_mask = nib.load('/Users/mahdi/stage-2022-mahdi/niftis/485847695_180403712_crossing_25/485847695_180403712_x-rois_mask_25.nii.gz')

In [4]:
template_arr = template.get_fdata()
xroi_mask_arr = xroi_mask.get_fdata()

In [14]:
xroi = (template_arr * xroi_mask_arr).clip(0,255).astype(np.uint8)

In [17]:
xroi_img = nib.Nifti1Image(xroi, affine)
nib.save(xroi_img, "/Users/mahdi/stage-2022-mahdi/niftis/xroi.nii.gz")